In [163]:
import math
import numpy as np

In [164]:
def sign(x):
    if x == 0:
        return 0
    else:
        return int(x / abs(x))

## Caesar Substitution Cipher

In [165]:
class CaesarSubstCipher():
    def __init__(self, substitution_key):
        self.substitution_key = substitution_key
        self.alphabet = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    
    def encrypt_message(self, raw_message):
        return "".join([self.substitution_key[self.alphabet.index(raw_message[i].upper())] for i in range(len(raw_message))])
    
    def decrypt_message(self, encrypted_message):
        return "".join([self.alphabet[self.substitution_key.index(encrypted_message[i].upper())] for i in range(len(encrypted_message))])

In [166]:
message = "HELLOBEAUTIFULPERSON"
subst_cipher = CaesarSubstCipher("EFGHIJKLMNOPQRSTUVWXYZABCD")
encrypted_message = subst_cipher.encrypt_message(message)
encrypted_message

'LIPPSFIEYXMJYPTIVWSR'

In [167]:
decrypted_message = subst_cipher.decrypt_message(encrypted_message)
decrypted_message

'HELLOBEAUTIFULPERSON'

## Caesar Permutation Cipher

In [168]:
class CaesarPermutationCipher():
    def __init__(self, substitution_key, permutation_key):
        self.substitution_key = substitution_key
        self.permutation_key = [int(character) for character in permutation_key]
        self.alphabet = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
        
    def _pad_message(self, message):
        if len(message) % len(self.permutation_key) != 0:
            return message + "X" * (len(self.permutation_key) - (len(message) % len(self.permutation_key)))
        else:
            return message
    
    def _permute_message(self, message):
        k = self.permutation_key
        message = self._pad_message(message)
        return "".join([message[(i // len(k))*len(k) + k[i % len(k)] - 1] for i in range(len(message))])
    
    def _depermute_message(self, message):
        perm_key = np.array(self.permutation_key) - 1
        permutation = np.append(perm_key, [perm_key + len(self.permutation_key)*i for i in range(1, len(message) // len(self.permutation_key))])
        return "".join([message[np.where(permutation==i)[0][0]] for i in range(len(message))])
        
    def encrypt_message(self, raw_message):
        raw_message = self._permute_message(raw_message)
        return "".join([self.substitution_key[self.alphabet.index(raw_message[i].upper())] for i in range(len(raw_message))])
    
    def decrypt_message(self, encrypted_message):
        encrypted_message = self._depermute_message(encrypted_message)
        return "".join([self.alphabet[self.substitution_key.index(encrypted_message[i].upper())] for i in range(len(encrypted_message))])

In [169]:
perm = CaesarPermutationCipher("EFGHIJKLMNOPQRSTUVWXYZABCD", "32541")

In [170]:
encrypted = perm.encrypt_message("HELLOBEAUTIFULPERSON")
encrypted

'PISPLEIXYFYJTPMWVRSI'

In [171]:
decrypted = perm.decrypt_message(encrypted)
decrypted

'HELLOBEAUTIFULPERSON'

## Vignete Cipher

In [172]:
class VigneteCipher():
    def __init__(self, keyword):
        self.keyword = keyword
        self.alphabet = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
        
    def encrypt_message(self, message):
        keyword = (self.keyword * (len(message) // len(self.keyword) + 1))[0:len(message)]
        encoded_as_nums = [(self.alphabet.index(message[i]) + self.alphabet.index(keyword[i])) % 26 
                           for i in range(len(message))]
        return "".join([self.alphabet[i] for i in encoded_as_nums])
    
    def decrypt_message(self, message):
        keyword = (self.keyword * (len(message) // len(self.keyword) + 1))[0:len(message)]
        encoded_as_nums = [(self.alphabet.index(message[i]) - self.alphabet.index(keyword[i])) % 26 
                           for i in range(len(message))]
        return "".join([self.alphabet[i] for i in encoded_as_nums])

In [173]:
vignete = VigneteCipher("LMFAO")
encrypted_message = vignete.encrypt_message("HELLOYOUAREBASED")
encrypted_message

'SQQLCJAZAFPNFSSO'

In [174]:
decrypted_message = vignete.decrypt_message(encrypted_message)
decrypted_message

'HELLOYOUAREBASED'

## Playfair Cipher

In [175]:
class PlayfairCipher():
    def __init__(self, keyword):
        self.keyword = keyword
        self.alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.table = []
        self.create_table()
        
    def create_table(self):
        self.table = []
        keyword = self.keyword + self.alphabet
        used_letters = ["J"]
        i = 0
#         print(self.table)
        while len(self.table) < 25:
            if keyword[i] in used_letters:
                pass
            else:
                used_letters.append(keyword[i])
                self.table.append(keyword[i])
            i+=1
        self.table = np.array(self.table).reshape((5, 5))
        
    def preprocess(self, text):
        k = len(text)
        if k % 2 == 0:
            for i in range(0, k, 2):
                if text[i] == text[i+1]:
                    new_word = text[0:i+1] + "X" + text[i+1:]
                    new_word = self.preprocess(new_word)
                    break
                else:
                    new_word = text
        else:
            for i in range(0, k-1, 2):
                if text[i] == text[i+1]:
                    new_word = text[0:i+1] + "X" + text[i+1:]
                    new_word = self.preprocess(new_word)
                    break
                else:
                    new_word = text
        return new_word + "X"*(len(new_word) % 2)
    
    def same_col(self, a, b, a_coords, b_coords):
        if a_coords[1] == b_coords[1]:
#             print("same col")
            return True
        else:
            return False
        
    def same_row(self, a, b, a_coords, b_coords):
        if a_coords[0] == b_coords[0]:
#             print("same row")
            return True
        else:
            return False
    
    def encrypt_message(self, message):
        encrypted_message = ""
        message = self.preprocess(message)
#         print("After preproc: " + message + " Len: " + str(len(message)))
        message = [message[i:i+2] for i in range(0, len(message), 2)]
        for pair in message:
            if pair[0] == "J":
                pair[0] = "I"
            if pair[1] == "J":
                pair[1] == "I"
                
            a = [int(item) for item in np.where(self.table == pair[0])] # coords of first letter
            b = [int(item) for item in np.where(self.table == pair[1])] # coords of second letter in pair
            
            is_same_col = self.same_col(pair[0], pair[1], a, b)
            is_same_row = self.same_row(pair[0], pair[1], a, b)
            if is_same_col:
                encrypted_message += self.table[(a[0] + 1) % 5, a[1]] + self.table[(b[0] + 1) % 5, b[1]]
            elif is_same_row:
                encrypted_message += self.table[a[0], (a[1] + 1) % 5] + self.table[b[0], (b[1] + 1) % 5]
            else:
#                 print("else")
                encrypted_message += self.table[a[0], b[1]] + self.table[b[0], a[1]]
        return encrypted_message
    
    def decrypt_message(self, message):
        encrypted_message = ""
        message = [message[i:i+2] for i in range(0, len(message), 2)]
        for pair in message:        
            a = [int(item) for item in np.where(self.table == pair[0])] # coords of first letter
            b = [int(item) for item in np.where(self.table == pair[1])] # coords of second letter in pair
            
            is_same_col = self.same_col(pair[0], pair[1], a, b)
            is_same_row = self.same_row(pair[0], pair[1], a, b)
            if is_same_col:
                encrypted_message += self.table[a[0] - 1, a[1]] + self.table[b[0] - 1, b[1]]
            elif is_same_row:
                encrypted_message += self.table[a[0], a[1] - 1] + self.table[b[0], b[1] - 1]
            else:
                encrypted_message += self.table[a[0], b[1]] + self.table[b[0], a[1]]
        return encrypted_message

In [176]:
playfair = PlayfairCipher("LULZ")

In [177]:
playfair.table, playfair.table[0, 2]

(array([['L', 'U', 'Z', 'A', 'B'],
        ['C', 'D', 'E', 'F', 'G'],
        ['H', 'I', 'K', 'M', 'N'],
        ['O', 'P', 'Q', 'R', 'S'],
        ['T', 'V', 'W', 'X', 'Y']], dtype='<U1'),
 'Z')

In [178]:
encrypted = playfair.encrypt_message("HELLOMYNAMEISRAZVANANDIAMASTUDENT")
encrypted

'KCATCTNXMBKFNPXFUWBMBMIPFRBRVLEFHY'

In [179]:
playfair.decrypt_message(encrypted)

'HELXLOMYNAMEISRAZVANANDIAMASTUDENT'